In [2]:
import pandas as pd
import random

In [3]:
csv = pd.read_csv('input/delitos_2016.csv')
for i in range(2017, 2021):
    csv = pd.concat([csv, pd.read_csv(f'input/delitos_{i}.csv')])

/var/folders/zk/0yzb9y8s6z71smfld5fjvdjh0000gn/T/ipykernel_24439/527257168.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  csv = pd.concat([csv, pd.read_csv(f'input/delitos_{i}.csv')])


In [4]:
# convert DD/MM/AAAA to DD-MM-AAAA
csv['fecha'] = csv['fecha'].apply(lambda x: x.replace('/', '-'))
# change lat/long to latitud/longitud
csv = csv.rename(columns={'lat': 'latitud', 'long': 'longitud'})
# if uso_armas is not present or null, set to "Sin Datos"
csv['uso_armas'] = csv['uso_armas'].fillna('Sin Datos')
# if subtipo is not present or null, set to "Sin Datos"
csv['subtipo'] = csv['subtipo'].fillna('Robo')
# remove id and id_mapa
csv = csv.drop(columns=['id', 'id_mapa', 'cantidad_registrada'])
# set anio, mes, dia with the fecha column
csv['anio'] = csv['fecha'].apply(lambda x: int(x.split('-')[2]))
csv['mes'] = csv['fecha'].apply(lambda x: int(x.split('-')[1]))
csv['nro_dia'] = csv['fecha'].apply(lambda x: int(x.split('-')[0]))
# set comuna to int
# csv['comuna'] = csv['comuna'].apply(lambda x: int(x))

In [5]:
csv.head()

,fecha,franja,tipo,subtipo,comuna,barrio,latitud,longitud,mes,dia,anio,uso_armas,victimas,nro_dia
0,01-01-2016,13,Robo (con violencia),Robo,9.0,Parque Avellaneda,-34.64873,-58.468536,1,Viernes,2016,Sin Datos,NaN,1
1,01-01-2016,22,Robo (con violencia),Robo,14.0,Palermo,-34.576925,-58.4074,1,Viernes,2016,Sin Datos,NaN,1
2,01-01-2016,1,Lesiones,Siniestro Vial,4.0,Parque Patricios,-34.645197,-58.401099,1,Viernes,2016,Sin Datos,NaN,1
3,01-01-2016,19,Hurto (sin violencia),Robo,9.0,Parque Avellaneda,-34.65576,-58.478681,1,Viernes,2016,Sin Datos,NaN,1
4,01-01-2016,4,Robo (con violencia),Robo Automotor,6.0,Caballito,-34.624128,-58.436826,1,Viernes,2016,Sin Datos,NaN,1


In [ ]:
# TABLA TIPOS
tipos_csv = csv[['tipo', 'subtipo']]
tipos_csv = tipos_csv.drop_duplicates(ignore_index=True)
tipos_csv = tipos_csv.fillna('Robo')
tipos_csv['id'] = tipos_csv.index + 1

In [ ]:
# TABLA TIEMPOS
tiempos_csv = csv[['anio', 'mes', 'dia', 'fecha', 'franja']]
tiempos_csv['nro_dia'] = tiempos_csv['fecha'].apply(lambda x: int(x.split('-')[0]))
tiempos_csv = tiempos_csv.drop_duplicates(ignore_index=True)
tiempos_csv['id'] = tiempos_csv.index + 1
tiempos_csv['dia'] = tiempos_csv['dia'].fillna('Sin Datos')
tiempos_csv

In [ ]:
# TABLA LUGARES
lugares_csv = csv[['barrio', 'comuna', 'latitud', 'longitud']]
# remove the ones that don't have latitud or longitud
lugares_csv = lugares_csv.dropna()
# convert latitud and longitud to numeric if they are not
lugares_csv['latitud'] = pd.to_numeric(lugares_csv['latitud'], errors='coerce')
lugares_csv['longitud'] = pd.to_numeric(lugares_csv['longitud'], errors='coerce')
lugares_csv = lugares_csv.groupby(['barrio', 'comuna']).max()
# if it is smaller than -75 it means there is an error, multiply it by 0.001
lugares_csv['latitud'] = lugares_csv['latitud'].apply(lambda x: x * 0.001 if x < -75 else x)
lugares_csv['longitud'] = lugares_csv['longitud'].apply(lambda x: x * 0.001 if x < -75 else x)
lugares_csv = lugares_csv.reset_index()
lugares_csv['id'] = lugares_csv.index + 1
lugares_csv

In [ ]:
# TABLA TIPO DE ARMA
armas_csv = csv[['uso_armas']]
armas_csv = armas_csv.drop_duplicates(ignore_index=True)
armas_csv['id'] = armas_csv.index + 1
armas_csv

In [ ]:
# TABLA DELITOS
delitos_csv = csv
delitos_csv

In [ ]:
# replace uso_armas for arma_id
delitos_csv = delitos_csv.merge(armas_csv, left_on=['uso_armas'], right_on=['uso_armas'])
delitos_csv = delitos_csv.rename(columns={'id': 'arma_id'})
delitos_csv

In [ ]:
# replace barrio for lugar_id
delitos_csv = delitos_csv.merge(lugares_csv, left_on=['barrio'], right_on=['barrio'])
delitos_csv = delitos_csv.rename(columns={'id': 'lugar_id'})
delitos_csv = delitos_csv.drop(columns=['barrio'])

In [ ]:
# replace tipo for tipo_id
delitos_csv = delitos_csv.merge(tipos_csv, left_on=['tipo', 'subtipo'], right_on=['tipo', 'subtipo'])
delitos_csv = delitos_csv.rename(columns={'id': 'tipo_id'})
delitos_csv = delitos_csv.drop(columns=['tipo', 'subtipo'])

In [ ]:
# # replace fecha for tiempo_id
delitos_csv = delitos_csv.merge(tiempos_csv, left_on=['fecha', 'franja'], right_on=['fecha', 'franja'])
delitos_csv = delitos_csv.rename(columns={'id': 'tiempo_id'})
delitos_csv = delitos_csv.drop(columns=['fecha'])

In [ ]:
# create a victimas column with random values
delitos_csv['victimas'] = delitos_csv.index
delitos_csv['costo'] = delitos_csv.index
delitos_csv['victimas'] = delitos_csv['victimas'].apply(lambda x: random.randint(0, 5))
delitos_csv['costo'] = delitos_csv['costo'].apply(lambda x: random.randint(0, 100000))

In [ ]:
delitos_csv = delitos_csv[['lugar_id', 'tiempo_id', 'arma_id', 'tipo_id', 'victimas', 'costo']]
delitos_csv

In [ ]:
# save csvs to files
tiempos_csv.to_csv('output/tiempos.csv', index=False)
lugares_csv.to_csv('output/lugares.csv', index=False)
armas_csv.to_csv('output/armas.csv', index=False)
delitos_csv.to_csv('output/delitos.csv', index=False)
tipos_csv.to_csv('output/tipos.csv', index=False)